## @api.depends(field1,field2)
- field可以为field.field样式
- field可以为function
    - 在这种情况下，通过调用带有字段模型的函数来给出依赖关系。
    - 个人认为是通过该函数返回与模型相关的字段
    
## @api.depends_context(field1,field2)
- 它是指定上下文中未存储的compute字段为依赖，如果上下文出现这个字段就会触发
- 这些依赖字段必须是可哈希的
        price = fields.Float(compute='_compute_product_price')

        @api.depends_context('pricelist')
        def _compute_product_price(self):
            for product in self:
                if product.env.context.get('pricelist'):
                    pricelist = self.env['product.pricelist'].browse(product.env.context['pricelist'])
                else:
                    pricelist = self.env['product.pricelist'].get_default_pricelist()
                product.price = pricelist.get_products_price(product).get(product.id, 0.0)

## @api.constraints(field1,field2)
- 字段约束：在记录集上调用这个方法，装饰器参数指定了约束涉及的字段，当涉及的字段中任一发生改变时触发方法执行，如果不满足约束条件，则该方法将引发异常
- 每个参数必须是用在检查中的字段，个人理解为即一定要使用这个参数
- 参数类型不支持field.field类型
- 只有当约束的字段在create或write方法中涉及时，才会调用这个装饰器，所以要保证在重写create方法时约束的字段会有所涉及

## @api.onchange([field1,field2])
- 在字段出现的表单视图中，当某个给定字段被修改时，该方法将被调用。或者某个伪记录（该记录未在数据库中进行创建）中含有这个字段时也会被调用
- onchange说明
    - 1.self是单条记录，在一个form里面显示
    - 2.方法里改变self的字段，会更新到未保存的form中，即前台数据在未保存就会自动跟新（主要用于相互关联的字段，例如单价，数量），也可以使用onchange判断多个字段的关系，并提示相关信息
    - 3.由于伪记录含该字段时也会调用该方法，但此时数据库中的数据并不存在，所以无法调用create,unlink,write,read方法。所以直接向字段赋值或者调用update方法
    - 4.与计算字段不同的是：
        - self是单条记录，不能放在tree view中
        - onchange方法的数据可以保存到数据中，计算字段内置了onchange行为，但是数据无法自动保存到数据库
    - 注：不能通过onchange来修改one2many\many2one\many2many字段的内容
- 其他：实时计算字段和onchange方法会自动在客户端调用，不需要在视图里做特别声明，但可以在视图中通过on_change="0"参数来阻止onchange函数的自动调用
- It is not possible for a one2many or many2many field to modify itself via onchange. This is a webclient limitation - see https://github.com/odoo/odoo/issues/2693.
        
## @api.returns(model, downgrade=None, upgrade=None)
- 返回模型的实例，即返回模型老api的使用
- 参数说明：
    - model – a model name, or 'self' for the current model
    - downgrade – 可以理解为将旧api的值转换为新api值的实现方法
    - upgrade – a function upgrade(self, value, *args, **kwargs) to convert the traditional-style value to a record-style output
    
## @api.model
- self本来应该是一个记录集的，但是经过该方法装饰以后，self就只代表模型本身，它不含任何的记录信息

## @api.model_create_multi
- 装饰一个采用字典列表并创建多个记录的方法。 可以使用单个字典或列表的形式调用该方法。
        @api.model_create_multi
        def create(self, vals_list):
            self._check_create()
            return super(ProductionLot, self).create(vals_list)

        record = model.create(vals)
        records = model.create([vals, ...])
        
## @api.ondelete(*, at_uninstall)
- 它是在调用unlink的方法时之前调用的，也就是说，通过该装饰器可以在unlink前执行某些操作，比如报错之类的，其实也可以直接在unlink中实现，但是在删除某些模块的时候会有冲突，所以该方法是为了解决这个问题
- 装饰的方法格式如下：
        `
        @api.ondelete(at_uninstall=False)
        def _unlink_if_user_inactive(self):
            if any(user.active for user in self):
                raise UserError("Can't delete an active user!")

        # same as above but with _unlink_except_* as method name
        @api.ondelete(at_uninstall=False)
        def _unlink_except_active_user(self):
            if any(user.active for user in self):
                raise UserError("Can't delete an active user!")
        `
        at_uninstall默认始终保持为False


## 模型约束
- odoo提供两种方式实现自动验证：python constraints和sql constraints
    - python约束通过方法constraints()来定义，并在记录集上调用这个方法，装饰器参数指定了约束涉及的字段，当涉及的字段中任一发生改变时触发方法执行，如果不满足约束条件，则该方法将引发异常
            from odoo.exceptions import ValidationError
            @api.constrains('age')
            def _check_age(self):
                for r in self:
                    if r.age > 20:
                        raise ValidationError("your age is too old")
    - sql constraints约束通过模型属性_sql_constraints进行定义，它是一个三元素的元组的列表(name,sql_definition,message)，其中name时sql的约束名称，sql_definition是约束规则，message是违反约束规则时的警告信息
    - 使用sql约束，必须时这个表中还没有创建数据才会生效，如果有数据，则约束并不会生效
    
            _sql_constraints = [
                ('name_description_check',
                 'CHECK(name !=description)',
                 "The title of the course should not be the description"),
                ('name_unique',
                 'UNIQUE(name)',
                 "The course title must be unique")
            ]

# 各类装饰器
## 记录集装饰器
- @api.one
    - 不是一个记录集，而self仅仅代表是一条数据，该装饰器无返回值
    - 在odoo14中应该已经把它删除了，默认使用的就是@api.multi
- @api.multi
    - self表示的是一个记录集，使用for循环对self进行循环迭代处理
    - 如果对某个方法不使用任何装饰器，那么会默认使用该装饰器
    - @api.one的功能可以使用@api.multi实现，使用self.ensure_one()即可
- @api.model
    - self不是一个记录集，而是一个对象，使用该装饰器的方法不能用于用户界面的按钮
    - 可以用来装饰定时任务
    
## 专有目的装饰器
- @api.depends(字段1，字段2)
    - 用于计算字段（computed field）,可用于标识需要被触发的计算
    - 需要注意计算字段必须被赋值
- @api.constraints(字段1，字段2）
    - 用于评估和检查，当参数字段的值发生变化时，会进行检查，如果检查规则不通过，则字段值不会发生变化，并会报出异常
- @api.onchange(字段1，字段2)
    - 用于系统在于用户进行交互时自动更新相关联的字段，就是当字段发生改变时，触发绑定的函数。
    - 仅在进行form视图交互的时候起作用
- @api.returns(
    - returns的用法主要是用来指定返回值的格式，它接受三个参数，第一个为返回值的model，第二个为向下兼容的method，第三个为向上兼容的method
- @api.autovacuum:类似自动清理垃圾的一个装饰器
        @api.autovacuum
        def _gc_profile(self):
            # remove profiles older than 30 days
            domain = [('create_date', '<', fields.Datetime.now() - datetime.timedelta(days=30))]
            return self.sudo().search(domain).unlink()